In [1]:
import tensorflow as tf
import pandas as pd
import numpy as np
from gensim.models import Word2Vec
import jieba  # 中文分词库，百度员工开发
import matplotlib.pyplot as plt 
import time
# 查询系统可用的 GPU
physical_devices = tf.config.experimental.list_physical_devices('GPU')
# 确保有可用的 GPU 如果没有, 则会报错
assert len(physical_devices) > 0, "Not enough GPU hardware devices available"
# 设置参数,该段务必在运行jupyter的第一段代码执行，否则会无法初始化成功
# 仅在需要时申请显存空间（程序初始运行时消耗很少的显存，随着程序的运行而动态申请显存）
tf.config.experimental.set_memory_growth(physical_devices[0], True)

#### 1.通过IMDB理解文本如何做one_hot编码

In [2]:
from tensorflow.keras.datasets import imdb
# 定义数据的本地导入
path_tmp = '/home/hp/.local/lib/python3.8/site-packages/tensorflow/keras/datasets/'
(train_data, train_labels), (test_data, test_labels) = imdb.load_data(path=path_tmp+'imdb.npz',
num_words=10000)


In [3]:
type(train_data)

numpy.ndarray

In [4]:
type(test_data)

numpy.ndarray

In [5]:
type(train_data[0])

list

In [6]:
# 查看训练数据前五个的数组长度
for i in range(5):
    print(len(train_data[i]))

218
189
141
550
147


In [7]:
# 查看第一个训练数据
print(train_data[0])

[1, 14, 22, 16, 43, 530, 973, 1622, 1385, 65, 458, 4468, 66, 3941, 4, 173, 36, 256, 5, 25, 100, 43, 838, 112, 50, 670, 2, 9, 35, 480, 284, 5, 150, 4, 172, 112, 167, 2, 336, 385, 39, 4, 172, 4536, 1111, 17, 546, 38, 13, 447, 4, 192, 50, 16, 6, 147, 2025, 19, 14, 22, 4, 1920, 4613, 469, 4, 22, 71, 87, 12, 16, 43, 530, 38, 76, 15, 13, 1247, 4, 22, 17, 515, 17, 12, 16, 626, 18, 2, 5, 62, 386, 12, 8, 316, 8, 106, 5, 4, 2223, 5244, 16, 480, 66, 3785, 33, 4, 130, 12, 16, 38, 619, 5, 25, 124, 51, 36, 135, 48, 25, 1415, 33, 6, 22, 12, 215, 28, 77, 52, 5, 14, 407, 16, 82, 2, 8, 4, 107, 117, 5952, 15, 256, 4, 2, 7, 3766, 5, 723, 36, 71, 43, 530, 476, 26, 400, 317, 46, 7, 4, 2, 1029, 13, 104, 88, 4, 381, 15, 297, 98, 32, 2071, 56, 26, 141, 6, 194, 7486, 18, 4, 226, 22, 21, 134, 476, 26, 480, 5, 144, 30, 5535, 18, 51, 36, 28, 224, 92, 25, 104, 4, 226, 65, 16, 38, 1334, 88, 12, 16, 283, 5, 16, 4472, 113, 103, 32, 15, 16, 5345, 19, 178, 32]


In [8]:
train_data[0].sort()

In [9]:
print(train_data[0])

[1, 2, 2, 2, 2, 2, 2, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 5, 5, 5, 5, 5, 5, 5, 5, 5, 6, 6, 6, 7, 7, 8, 8, 8, 9, 12, 12, 12, 12, 12, 12, 13, 13, 13, 14, 14, 14, 15, 15, 15, 15, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 17, 17, 17, 18, 18, 18, 19, 19, 21, 22, 22, 22, 22, 22, 22, 25, 25, 25, 25, 26, 26, 26, 28, 28, 30, 32, 32, 32, 33, 33, 35, 36, 36, 36, 36, 38, 38, 38, 38, 39, 43, 43, 43, 43, 46, 48, 50, 50, 51, 51, 52, 56, 62, 65, 65, 66, 66, 71, 71, 76, 77, 82, 87, 88, 88, 92, 98, 100, 103, 104, 104, 106, 107, 112, 112, 113, 117, 124, 130, 134, 135, 141, 144, 147, 150, 167, 172, 172, 173, 178, 192, 194, 215, 224, 226, 226, 256, 256, 283, 284, 297, 316, 317, 336, 381, 385, 386, 400, 407, 447, 458, 469, 476, 476, 480, 480, 480, 515, 530, 530, 530, 546, 619, 626, 670, 723, 838, 973, 1029, 1111, 1247, 1334, 1385, 1415, 1622, 1920, 2025, 2071, 2223, 3766, 3785, 3941, 4468, 4472, 4536, 4613, 5244, 5345, 5535, 5952, 7486]


In [10]:
train_data.shape

(25000,)

In [11]:
# 查看标签数据前五个的数组长度
train_labels[0:5]

array([1, 0, 0, 1, 0])

In [12]:
# 导入单词索引,是一个字典形式
word_index = imdb.get_word_index(path=path_tmp+'imdb_word_index.json')

In [13]:
type(word_index)

dict

In [14]:
# inverse_word_index ={}
# for key,val in word_index.items():
#     inverse_word_index[val] = key

# # 按照值进行排序
# inverse_word_index = sorted(inverse_word_index.items(), key = lambda k: k[0])

# # 重新存为一个新字典
# new_word_index = {}
# for i in inverse_word_index:
#     new_word_index[i[0]] =i[1] 

In [15]:
# 查看字典的前几个索引对应的单词
for key,val in word_index.items():
    if val < 5:
        print(key,',',val)

of , 4
a , 3
the , 1
and , 2


In [16]:
reverse_word_index = dict(
    (value, key) for (key, value) in word_index.items()
)

In [17]:
decoded_review_0 = ' '.join(
    reverse_word_index.get(i - 3, '?') for i in train_data[0]
)

In [18]:
# 查看第一条评论，通过翻译可知，这是一条正面积极的评论，label为1
decoded_review_0

"? ? ? ? ? ? ? the the the the the the the the the the the the the the the and and and and and and and and and a a a of of to to to is it it it it it it i i i this this this that that that that was was was was was was was was was was was as as as for for for with with but film film film film film film you you you you are are are have have be all all all at at an they they they they so so so so from just just just just out if there there what what good up would story story really really were were much been also great because because don't them could after think think watch two being being life little know end these say such should real now director same same part us fact big must done whole whole played played true actor play everyone left father stars came recommend often definitely loved direction throughout children children amazing amazing amazing soon brilliant brilliant brilliant myself sad released robert paul imagine casting list island bought lovely scenery cry location witty c

In [19]:
decoded_review_1 = ' '.join(
    reverse_word_index.get(i - 3, '?') for i in train_data[1]
)

In [20]:
# 查看第二条评论可以知道，这是一条负面的评论，label为0
decoded_review_1

"? big hair big boobs bad music and a giant safety pin these are the words to best describe this terrible movie i love cheesy horror movies and i've seen hundreds but this had got to be on of the worst ever made the plot is paper thin and ridiculous the acting is an abomination the script is completely laughable the best is the end showdown with the cop and how he worked out who the killer is it's just so damn terribly written the clothes are sickening and funny in equal ? the hair is big lots of boobs ? men wear those cut ? shirts that show off their ? sickening that men actually wore them and the music is just ? trash that plays over and over again in almost every scene there is trashy music boobs and ? taking away bodies and the gym still doesn't close for ? all joking aside this is a truly bad film whose only charm is to look back on the disaster that was the 80's and have a good old laugh at how bad everything was back then"

In [21]:
 random_test = np.random.random((3,2))

In [22]:
random_test

array([[0.27592385, 0.72894211],
       [0.03599346, 0.92322249],
       [0.00794755, 0.98331935]])

In [23]:
random_test[1,[0,1,1]] # 取第二行的第1个和第2个数据,这个位置方便理解下面的vectorize_sequences函数

array([0.03599346, 0.92322249, 0.92322249])

In [24]:
random_test[1,[0,1]] = 1.

In [25]:
random_test

array([[0.27592385, 0.72894211],
       [1.        , 1.        ],
       [0.00794755, 0.98331935]])

In [26]:
random_test[0,0]

0.275923854027733

In [27]:
#　使用one_hot编码的形式对整数列表转换为张量,
def vectorize_sequences(sequences,dimension = 10000):
    '''
    函数名：向量化序列
    sequences： 输入列表
    dimension：维度
    '''
    results = np.zeros((len(sequences), dimension)) # 创建一个序列数据样本数等同的行数，10000列数的0矩阵
    for i, sequence in enumerate(sequences):
        results[i, sequence] = 1.  # 有数据的地方标注为0,其他取不出数据的地方默认为0不变
    return results

In [28]:
for i, sequence in enumerate(train_data[0:1]):
    print(i, '\n', sequence)

0 
 [1, 2, 2, 2, 2, 2, 2, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 5, 5, 5, 5, 5, 5, 5, 5, 5, 6, 6, 6, 7, 7, 8, 8, 8, 9, 12, 12, 12, 12, 12, 12, 13, 13, 13, 14, 14, 14, 15, 15, 15, 15, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 17, 17, 17, 18, 18, 18, 19, 19, 21, 22, 22, 22, 22, 22, 22, 25, 25, 25, 25, 26, 26, 26, 28, 28, 30, 32, 32, 32, 33, 33, 35, 36, 36, 36, 36, 38, 38, 38, 38, 39, 43, 43, 43, 43, 46, 48, 50, 50, 51, 51, 52, 56, 62, 65, 65, 66, 66, 71, 71, 76, 77, 82, 87, 88, 88, 92, 98, 100, 103, 104, 104, 106, 107, 112, 112, 113, 117, 124, 130, 134, 135, 141, 144, 147, 150, 167, 172, 172, 173, 178, 192, 194, 215, 224, 226, 226, 256, 256, 283, 284, 297, 316, 317, 336, 381, 385, 386, 400, 407, 447, 458, 469, 476, 476, 480, 480, 480, 515, 530, 530, 530, 546, 619, 626, 670, 723, 838, 973, 1029, 1111, 1247, 1334, 1385, 1415, 1622, 1920, 2025, 2071, 2223, 3766, 3785, 3941, 4468, 4472, 4536, 4613, 5244, 5345, 5535, 5952, 7486]


In [29]:
##### 相当于在每一行数据中都是一个one_hot编码

In [30]:
x_train = vectorize_sequences(train_data)
x_test = vectorize_sequences(test_data)

In [31]:
x_train[0]

array([0., 1., 1., ..., 0., 0., 0.])

In [32]:
x_train.shape

(25000, 10000)

In [33]:
y_train = np.asarray(train_labels).astype('float32')
y_test = np.asarray(test_labels).astype('float32')

In [34]:
train_labels

array([1, 0, 0, ..., 0, 1, 0])

In [35]:
y_train

array([1., 0., 0., ..., 0., 1., 0.], dtype=float32)

In [36]:
type(y_train)

numpy.ndarray

In [37]:
x_train_temp =  vectorize_sequences(train_data[0])

In [38]:
x_train_temp.shape

(218, 10000)

In [39]:
x_train_temp

array([[0., 1., 0., ..., 0., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [40]:
x_train_temp

array([[0., 1., 0., ..., 0., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [41]:
# 还需要自己建造一个短的数组来理解这个问题
train_sample = [
    [23,14,22,11],
    [8,1,2,11],
    [4,3,4,4],
    [3,1,22,11,2],
]

In [42]:
# dimension的设置必须大于train_sample中最大的数
x_train_sample = vectorize_sequences(train_sample, dimension = 30)

In [43]:
x_train_sample

array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 1., 0.,
        0., 0., 0., 0., 0., 0., 1., 1., 0., 0., 0., 0., 0., 0.],
       [0., 1., 1., 0., 0., 0., 0., 0., 1., 0., 0., 1., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 1., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 1., 1., 1., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0.]])

In [44]:
train_sample[2]

[4, 3, 4, 4]

In [45]:
x_train_sample[2]

array([0., 0., 0., 1., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])

##### 实验证明，这样的处理方式就会导致如果存在一个词语若在一个句子中存在重复单词的情况，数据处理后只会按照一个单词位置存储，不会体现词语的重复性特征

该点存疑，后期再回顾确认？？？

#### 2.通过文本更加深入理解one_hot编码

In [47]:
samples = ['the cat sat on the mat.', 'the dog ate my homework.']

In [53]:
samples[0].split()

['the', 'cat', 'sat', 'on', 'the', 'mat.']

In [48]:
token_index = {}
for sample in samples:
    for word in sample.split(): # 默认用空格分词，实际应用中还需要去掉标点和特殊字符
        if word not in token_index:
            token_index[word] = len(token_index) + 1 # 为每一个唯一单词指定一个唯一索引，注意索引0未指定单词

In [54]:
token_index

{'the': 1,
 'cat': 2,
 'sat': 3,
 'on': 4,
 'mat.': 5,
 'dog': 6,
 'ate': 7,
 'my': 8,
 'homework.': 9}

In [57]:
max_length = 10

In [79]:
# 生成一个张量的示例
np.zeros(shape = (2,2,3))

array([[[0., 0., 0.],
        [0., 0., 0.]],

       [[0., 0., 0.],
        [0., 0., 0.]]])

In [77]:
results = np.zeros(
    shape = (len(samples),
            max_length,
            max(token_index.values()) + 1 )
                    )

In [80]:
for i, sample in enumerate(samples):
    for j , word in list(enumerate(sample.split()))[:max_length]:
        index = token_index.get(word)
        results[i, j, index] = 1.

In [100]:
 # i来表示这个长句子所处的张量的位置，j表示这个张量的横轴位置，index表示这个张量的纵轴位置，通过这样映射关系来确定单词在one_hot中的位置
 results.shape

(2, 10, 10)

In [81]:
results

array([[[0., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 1., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 1., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 1., 0., 0., 0., 0., 0.],
        [0., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 1., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]],

       [[0., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 1., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 1., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 1., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 1.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]

In [92]:
results[0,[0]]  # the

array([[0., 1., 0., 0., 0., 0., 0., 0., 0., 0.]])

In [93]:
results[0,[1]] # cat

array([[0., 0., 1., 0., 0., 0., 0., 0., 0., 0.]])

In [94]:
results[0,[2]] # sat

array([[0., 0., 0., 1., 0., 0., 0., 0., 0., 0.]])

In [95]:
results[0,[3]] # on

array([[0., 0., 0., 0., 1., 0., 0., 0., 0., 0.]])

In [96]:
results[0,[4]] # the

array([[0., 1., 0., 0., 0., 0., 0., 0., 0., 0.]])

In [97]:
results[0,[5]] # mat

array([[0., 0., 0., 0., 0., 1., 0., 0., 0., 0.]])

In [99]:
results[0,[6]] # 从6-9都是全部填充为0

array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]])

##### 总结和需要注意的点
- 构建一个固定长度的张量，对每一个单词都映射一个one_hot编码来表示单词，重复的地方仍然重复表示
- 如果缺位的地方，是全部用0填充
- 这个方法有个缺点，就是当词表中的单词唯一值过大，会导致编码的张量横轴维度很大

#### 3.通过散列编码解决one_hot的单词唯一值太大问题
- 散列编码实质上是对一个单词求一个固定的hash长度，然后对这个长度除以张量的维度求余，这样的求余数可能会导致不同单词的散列编码相同

In [101]:
samples = ['the cat sat on the mat.', 'the dog ate my homework.']

In [102]:
dimensionality = 1000 # 定义单词保存长度为1000，如果单词数据接近1000个，那么会遭遇散列冲突，就是不同的单词可能由同一个散列值表示
max_length = 10


In [104]:
results = np.zeros((len(samples), max_length, dimensionality))

In [111]:
# 哈希（hash）也翻译作散列。Hash算法，是将一个不定长的输入，通过散列函数变换成一个定长的输出，即散列值
 
# hash函数可以生成任意字符串或者数值的哈希值，且当更换环境后（比如换一个jupyter），同一个字符串的hash将改变
hash('test')

8533866751935312883

In [112]:
len(str(hash('test')))

19

In [113]:
len(str(hash('test1')))

20

In [116]:
20 % 1000

20

In [119]:
10020 % 1000

20

In [105]:
for i, sample in enumerate(samples):
    for j, word in list(enumerate(sample.split()))[:max_length]:
        index = abs(hash(word)) % dimensionality #求余数，这样就可以取到一个相近的表示
        results[i, j, index] = 1.

In [114]:
results

array([[[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]],

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]]])

In [115]:
results.shape

(2, 10, 1000)

#### 4.通过Embedding词嵌入方法将单词转换为低维浮点数向量
- 第一步的做法是统计所有的单词的出现频率，并按照频率做降序排序，然后选择TOP N个常见单词，如N = 10000，其他的剔除
- 将每一条文本的单词长度限定为M，M∈N，例如 'i love you so much'只选择'i love you'（因为so much并不在TOP N中）
- 实例化M的长度为20，则'i love you'缺失的长度全部由0填充，如果一条文本的长度超出20，则做截断处理

In [121]:
from tensorflow.keras.datasets import imdb
from tensorflow.keras import preprocessing
max_features = 10000 # 定义TOP N
maxlen = 20 # 定义M

In [124]:
# 定义数据的本地导入
(x_train, y_train), (x_test, y_testb) = imdb.load_data(path=path_tmp+'imdb.npz',
num_words=10000)

In [126]:
print(x_train[0])

[1, 14, 22, 16, 43, 530, 973, 1622, 1385, 65, 458, 4468, 66, 3941, 4, 173, 36, 256, 5, 25, 100, 43, 838, 112, 50, 670, 2, 9, 35, 480, 284, 5, 150, 4, 172, 112, 167, 2, 336, 385, 39, 4, 172, 4536, 1111, 17, 546, 38, 13, 447, 4, 192, 50, 16, 6, 147, 2025, 19, 14, 22, 4, 1920, 4613, 469, 4, 22, 71, 87, 12, 16, 43, 530, 38, 76, 15, 13, 1247, 4, 22, 17, 515, 17, 12, 16, 626, 18, 2, 5, 62, 386, 12, 8, 316, 8, 106, 5, 4, 2223, 5244, 16, 480, 66, 3785, 33, 4, 130, 12, 16, 38, 619, 5, 25, 124, 51, 36, 135, 48, 25, 1415, 33, 6, 22, 12, 215, 28, 77, 52, 5, 14, 407, 16, 82, 2, 8, 4, 107, 117, 5952, 15, 256, 4, 2, 7, 3766, 5, 723, 36, 71, 43, 530, 476, 26, 400, 317, 46, 7, 4, 2, 1029, 13, 104, 88, 4, 381, 15, 297, 98, 32, 2071, 56, 26, 141, 6, 194, 7486, 18, 4, 226, 22, 21, 134, 476, 26, 480, 5, 144, 30, 5535, 18, 51, 36, 28, 224, 92, 25, 104, 4, 226, 65, 16, 38, 1334, 88, 12, 16, 283, 5, 16, 4472, 113, 103, 32, 15, 16, 5345, 19, 178, 32]


In [127]:
# 将整数列表提取为长度只有20的关键词张量
x_train = preprocessing.sequence.pad_sequences(x_train,maxlen = maxlen)
x_test = preprocessing.sequence.pad_sequences(x_test,maxlen = maxlen)

In [128]:
print(x_train[0])

[  65   16   38 1334   88   12   16  283    5   16 4472  113  103   32
   15   16 5345   19  178   32]


In [130]:
x_train[0].shape

(20,)

In [129]:
x_train.shape

(25000, 20)

##### 使用Embedding层和分类器

In [131]:
# Flatten层用来将输入“压平”，即把多维的输入一维化
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten, Dense, Embedding

In [135]:
model = Sequential()
model.add(Embedding(10000, 8, input_length=maxlen)) #指定Embedding层的最大输入长度，以便后面将嵌入输入展平，Embedding层激活的形状为（samples, maxlen, 8)

In [136]:
model.add(Flatten()) # 将三维的嵌入张量展平为形状为（samples, maxlen * 8）的二维张量

In [137]:
model.add(Dense(1, activation='sigmoid')) # 在上面添加分类器

In [138]:
model.compile(optimizer='rmsprop', loss = 'binary_crossentropy', metrics=['acc'])
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 20, 8)             80000     
_________________________________________________________________
flatten (Flatten)            (None, 160)               0         
_________________________________________________________________
dense (Dense)                (None, 1)                 161       
Total params: 80,161
Trainable params: 80,161
Non-trainable params: 0
_________________________________________________________________


In [140]:
history = model.fit(x_train, y_train, epochs=10, batch_size=32, validation_split=0.2)

Epoch 1/10
625/625 [==============================] - 2s 4ms/step - loss: 0.2917 - acc: 0.8816 - val_loss: 0.5270 - val_acc: 0.7488
Epoch 2/10
625/625 [==============================] - 2s 4ms/step - loss: 0.2759 - acc: 0.8895 - val_loss: 0.5340 - val_acc: 0.7448
Epoch 3/10
625/625 [==============================] - 2s 4ms/step - loss: 0.2604 - acc: 0.8971 - val_loss: 0.5431 - val_acc: 0.7414
Epoch 4/10
625/625 [==============================] - 2s 4ms/step - loss: 0.2452 - acc: 0.9050 - val_loss: 0.5535 - val_acc: 0.7400
Epoch 5/10
625/625 [==============================] - 2s 4ms/step - loss: 0.2310 - acc: 0.9118 - val_loss: 0.5647 - val_acc: 0.7376
Epoch 6/10
625/625 [==============================] - 2s 4ms/step - loss: 0.2174 - acc: 0.9193 - val_loss: 0.5775 - val_acc: 0.7340
Epoch 7/10
625/625 [==============================] - 2s 4ms/step - loss: 0.2047 - acc: 0.9251 - val_loss: 0.5880 - val_acc: 0.7342
Epoch 8/10
625/625 [==============================] - 2s 4ms/step - loss: 0.